<a href="https://colab.research.google.com/github/stsushimaHTB/test/blob/master/%E8%83%8C%E6%99%AF%E7%94%BB%E5%83%8F%E3%81%AB%E8%A8%AD%E5%95%8F%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E3%82%92%E5%90%88%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **背景画像に設問テキストを合成する**

1.   GoogleDriveへ接続するための認証処理を実行
2.   スプレッドシートへ接続するための認証処理を実行



In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

!cp -a "gdrive/My Drive/Colab Notebooks/作画/font" "/usr/share/fonts/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

# 認証処理
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

# スプレッドシートキーを指定してワークブックを選択
workbook = gc.open_by_key('13c9yuIBeClJFmnMd5DLEPSN0i7UGzXZkjBsozn2iCPI')

# ワークシート名を直接指定
worksheet = workbook.worksheet('結果')

# 行番号と列番号を指定してセルの値を取得する（左：行番号、右：列番号）
question = worksheet.cell(1, 2).value

print(question)

これはきっと存在する！


In [3]:
def pillow_char_offset(x_pos: int, y_pos: int, x_end_pos: int, y_end_pos: int, char: str, init_font_size: int, save_pass: str):
    '''
    :param x_pos 文字を描画したい場所の[X]軸始点
    :param y_pos 文字を描画したい場所の[Y]軸始点
    :param x_end_pos 文字を描画したい場所の[X]軸終点
    :param y_end_pos 文字を描画したい場所の[Y]軸終点
    :param char 描画したい文字
    :param init_font_size フォントの初期値(最大サイズ)
    :param fontfile_name フォントファイル(otfやttf)のパス
    '''
    # ベースとなる画像の下地を作成する。複製した画像などをベースにする場合は Image.open で開く
    img = Image.new('L', (x_end_pos, y_end_pos), 'white')
    draw = ImageDraw.Draw(img)

    img.save('/content/gdrive/My Drive/Colab Notebooks/作画/hogehoge.jpg')

    length = x_end_pos - x_pos
    height = y_end_pos - y_pos
    out_text_size = (length + 1, height + 1)
    font_size_offset = 0
    font = None
    fontfile_name = "/usr/share/fonts/font/MEIRYOB.TTC" #ここでフォント指定

    # フォントの描画サイズが描画領域のサイズを下回るまでwhile
    while length < out_text_size[0] or height < out_text_size[1]:
        font = ImageFont.truetype(fontfile_name, init_font_size - font_size_offset)
        # draw.textsizeで描画時のサイズを取得
        out_text_size = draw.textsize(char, font=font)
        font_size_offset += 1

    draw.text((x_pos, y_pos), char, fill='#000', font=font)

    #img.save('/content/gdrive/My Drive/Colab Notebooks/作画/hogehoge_def.jpg')
    img.save(save_pass)


In [4]:
from PIL import Image, ImageDraw, ImageFont

pillow_char_offset(0, 0, 1055, 150, question, 130, '/content/gdrive/My Drive/Colab Notebooks/作画/todays_Q.jpg')

im_b = Image.open('/content/gdrive/My Drive/Colab Notebooks/作画/背景画像_途中経過_サイズ大_サイズ変更.jpg')
im_t = Image.open('/content/gdrive/My Drive/Colab Notebooks/作画/todays_Q.jpg')

back_im = im_b.copy()
back_im.paste(im_t, (750, 280))

back_im.save('/content/gdrive/My Drive/Colab Notebooks/作画/合成_背景画像.jpg', quality=95)